# GHG Emissions Implementation

## Imports

In [1]:
import os
import pandas as pd
import numpy as np

## Global Variables

In [2]:
COLAB = True

In [3]:
ROOT_DIR_PATH = os.path.abspath('..')

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

  ROOT_DIR_PATH = os.path.abspath('drive/MyDrive/Spatial_Finance_Transport/minorRoads/')

ROOT_VEHICLE_DETECTION_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/vehicle_counts/vehicle_counts_')
EMISSIONS_FACTORS_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/uk_emissions_factors.csv')
VEHICLE_COUNTS_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/vehicle_counts/')

COUNT_SITE_PATHS = ['luton_m1_2557A.csv', 'luton_m1_2557B.csv', 'havering_m25_5790A.csv', 'havering_m25_5790B.csv', 
                           'hounslow_m4_2188A.csv', 'hounslow_m4_2188B.csv',
                           'blackburn_30361033.csv', 'blackburn_30361032.csv']

AADT_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/aadt/')
GHG_EMISSIONS_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/ghg_emissions/')

Mounted at /content/drive


In [4]:
VEHICLE_CATEGORIES = ['Passenger Vehicle',
  'Small Car',
  'Bus',
  'Pickup Truck',
  'Utility Truck',
  'Truck',
  'Cargo Truck',
  'Truck w/Box',
  'Truck Tractor',
  'Trailer',
  'Truck w/Flatbed',
  'Truck w/Liquid',
  'Passenger Car'
]

EMISSIONS_CATEGORY_MAPPING = {
    'Passenger Vehicle': 'Petrol cars',
    'Small Car': 'Petrol cars',
    'Pickup Truck': 'Petrol LGVs',
    'Utility Truck': 'Petrol LGVs',
    'Truck': 'Petrol LGVs',
    'Cargo Truck': 'Rigid HGVs',
    'Truck Tractor': 'Rigid HGVs',
    'Trailer': 'Petrol LGVs',
    'Truck w/Flatbed': 'Rigid HGVs',
    'Truck w/Liquid': 'Rigid HGVs',
    'Passenger Car': 'Petrol cars',
    'Truck w/Box': 'Petrol LGVs',
    'Bus': 'Buses',
    'Trailer': 'Petrol LGVs',
    'Cargo Car': 'Petrol LGVs'
}

VEHICLE_KM_PER_LITRE_MAPPING = {
    'aadt': 10,
    'cars_and_taxis': 20,
    'buses_and_coaches': 3,
    'lgvs': 5,
    'all_hgvs': 5
}

VEHICLE_EMISSIONS_FACTORS_MAPPING = {
    'aadt': 'Petrol cars',
    'cars_and_taxis': 'Petrol cars',
    'buses_and_coaches': 'Buses',
    'lgvs': 'Petrol LGVs',
    'all_hgvs': 'Rigid HGVs'
}

KG_TO_KT = 1e-6

In [5]:
# km
LUTON_ROAD_LENGTH = 429.8
BLACKBURN_ROAD_LENGTH = 494.6
HOUNSLOW_ROAD_LENGTH = 419.5
HAVERING_ROAD_LENGTH = 597.9
TRAFFORD_ROAD_LENGTH = 734.8

## Helper Functions

In [6]:
def save_float_to_csv(float_value, column_name, image_id, file_name):
    """
    Save a float value to a CSV file with the specified column name and file name.
    
    Args:
        float_value (float): The float value to be saved.
        column_name (str): The name of the column in the CSV file.
        file_name (str): The name of the CSV file to be saved.
    """
    # Create a DataFrame with a single row and the specified column name and value
    df = pd.DataFrame({'image_id': image_id, column_name: [float_value]})
    
    # Save the DataFrame to a CSV file
    df.to_csv(file_name, index=False)

In [7]:
def get_files_in_directory(directory):
    """
    Get a list of all files in a directory.

    Args:
        directory (str): Directory path.

    Returns:
        list: List of files in the directory.
    """
    files = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            files.append(file_path)
    return files

In [8]:
def convert_category_names(dataframes_list, mapping_dict):
    """
    Convert category names in a list of DataFrames using a mapping dictionary.

    Args:
        dataframes_list (list): List of DataFrames with 'category_name' column.
        mapping_dict (dict): Dictionary containing mapping of old category names to new category names.

    Returns:
        list: List of DataFrames with updated category names.
    """
    updated_dataframes = []
    for df in dataframes_list:
        df['Vehicle Type'] = df['category_name'].map(mapping_dict)
        updated_dataframes.append(df)
    return updated_dataframes

In [9]:
def calculate_ghg_emissions(car_category, emission_factors, road_length):
    """
    Calculates GHG emissions for a given car category, emission factors, and road length.
    
    Args:
        car_category (str): Category of the car (e.g., "Small Car", "Midsize Car", etc.).
        emission_factors (dict): Dictionary containing emission factors for different car categories.
        road_length (float): Length of the road segment in kilometers.
        
    Returns:
        float: Total GHG emissions in kilograms for the given car category and road length.
    """
    # Check if the emission factors dictionary contains the given car category
    if car_category not in emission_factors:
        raise ValueError("Car category not found in emission factors dictionary.")
    
    # Get the emission factors for the given car category
    car_emission_factors = emission_factors[car_category]
    
    # Calculate GHG emissions using the emission factors and road length
    ghg_emissions = car_emission_factors['co2'] * road_length + \
                    car_emission_factors['ch4'] * road_length + \
                    car_emission_factors['n2o'] * road_length
    
    return ghg_emissions

In [10]:
def add_total_column(dataframes_list, other_dataframe):
    """
    Add a 'Total' column from one DataFrame to each DataFrame in a list of DataFrames based on the 'Vehicle Type' column.

    Args:
        dataframes_list (list): List of DataFrames.
        other_dataframe (DataFrame): DataFrame to extract the 'Total' column from.

    Returns:
        list: List of DataFrames with the 'Total' column added.
    """
    updated_dataframes = []
    for df in dataframes_list:
        if 'Vehicle Type' in df.columns and 'Vehicle Type' in other_dataframe.columns:
            total_column = other_dataframe[['Vehicle Type', 'Total']]
            df = df.merge(total_column, on='Vehicle Type', how='left')
        updated_dataframes.append(df)
    return updated_dataframes


In [11]:
def create_vehicle_type_counts_df(df):
    """
    Count unique vehicle types in a DataFrame and return a DataFrame with columns as vehicle types and
    a single row with counts as values.
    
    Args:
        df (pandas.DataFrame): DataFrame containing the columns: image_id, x_min, x_max, y_min, y_max,
                               category_name, area, Vehicle Type, and Total.
                               
    Returns:
        pandas.DataFrame: DataFrame with columns as vehicle types and a single row with counts as values.
    """
    # Check if "Vehicle Type" column is present in the DataFrame
    if "Vehicle Type" not in df.columns:
        raise ValueError("Column 'Vehicle Type' not found in the DataFrame.")
    
    # Count unique values in "Vehicle Type" column
    vehicle_type_counts = df["Vehicle Type"].value_counts().to_dict()
    
    # Create a DataFrame from the counts dictionary
    counts_df = pd.DataFrame(vehicle_type_counts, index=[0])
    
    return counts_df

## Load Vehicle Detection Data

In [12]:
dfs = []

vehicle_count_paths = get_files_in_directory(VEHICLE_COUNTS_ROOT_PATH)

for vehicle_count_path in vehicle_count_paths:
  df = pd.read_csv(vehicle_count_path)

  df.name = df.iloc[0]['image_id']
  print(df.name)
  dfs.append(df)

dfs[1].head()

blackburn_30361032
blackburn_30361033
havering_m25_5790a
havering_m25_5790b
hounslow_m4_2188a
hounslow_m4_2188b
trafford_m60_9083a
trafford_m60_9086b
luton_m1_2557a
luton_m1_2557b


,image_id,x_min,x_max,y_min,y_max,category_name,area
0,blackburn_30361033,1989.843464,2006.665344,1130.735352,1133.000000,Small Car,38
1,blackburn_30361033,1920.000000,1934.552798,1130.633972,1133.000000,Small Car,34
2,blackburn_30361033,768.000000,782.847470,1130.753998,1133.000000,Small Car,33
3,blackburn_30361033,1536.000000,1550.317162,1130.669983,1133.000000,Small Car,33
4,blackburn_30361033,1804.943848,1821.618256,1130.752686,1132.964996,Small Car,36


## Load Emissions Data

In [13]:
df_emissions_factors = pd.read_csv(EMISSIONS_FACTORS_PATH)

df_emissions_factors['Total'] = df_emissions_factors.sum(axis=1)

df_emissions_factors

<ipython-input-13-77218a245191>:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_emissions_factors['Total'] = df_emissions_factors.sum(axis=1)


,Vehicle Type,NOx,PM10,PM2.5,CO,VOC,NH3,SO2,Benzene,N2O,Total
0,Petrol cars,0.065000,0.001000,0.001000,0.5830,0.090000,0.013000,0.00000,0.002000,0.001000,0.756000
1,Diesel cars,0.517000,0.008000,0.008000,0.0470,0.004000,0.004000,0.00100,0.000000,0.006000,0.595000
2,Petrol LGVs,0.094000,0.001000,0.001000,1.2340,0.085000,0.017000,0.00100,0.002000,0.002000,1.437000
3,Diesel LGVs,0.808000,0.007000,0.007000,0.0580,0.008000,0.005000,0.00100,0.000000,0.006000,0.900000
4,Rigid HGVs,1.428000,0.020000,0.020000,0.4380,0.039000,0.009000,0.00200,0.000000,0.031000,1.987000
5,Artic HGVs,0.609000,0.011000,0.011000,0.2950,0.027000,0.009000,0.00300,0.000000,0.052000,1.017000
6,Buses,2.602000,0.030000,0.030000,0.7530,0.056000,0.008000,0.00300,0.000000,0.032000,3.514000
7,M/cycle,0.078934,0.007047,0.007047,2.3038,0.280368,0.001973,0.00036,0.012611,0.001822,2.693961


## Convert Vehicle Detection into Compatible Emissions Categories

In [14]:
dfs = convert_category_names(dfs, EMISSIONS_CATEGORY_MAPPING)

dfs[0].head()

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type
0,blackburn_30361032,1067.783325,1079.664490,1027.531982,1036.002106,Small Car,100,Petrol cars
1,blackburn_30361032,1975.399071,1986.786041,528.676117,538.260757,Small Car,109,Petrol cars
2,blackburn_30361032,1979.620247,1991.101990,514.587952,524.379181,Small Car,112,Petrol cars
3,blackburn_30361032,1824.158295,1833.719086,641.948860,650.265575,Small Car,79,Petrol cars
4,blackburn_30361032,2237.735962,2248.019989,209.058350,219.792877,Small Car,110,Petrol cars


## AADT by Number of Vehicles

### Load AADT data

In [15]:
aadt_paths = get_files_in_directory(AADT_ROOT_PATH)

for aadt_path in aadt_paths:
  df_aadt = pd.read_csv(aadt_path, sep = ',', skipinitialspace = True)

  for df in dfs:
    if df.iloc[0]['image_id'] == df_aadt.iloc[0]['image_id']:
      df['aadt'] = df_aadt.iloc[0]['aadt']
      df['cars_and_taxis'] = df_aadt.iloc[0]['cars_and_taxis']
      df['buses_and_coaches'] = df_aadt.iloc[0]['buses_and_coaches']
      df['lgvs'] = df_aadt.iloc[0]['lgvs']
      df['all_hgvs'] = df_aadt.iloc[0]['all_hgvs']

dfs[0].head()

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs
0,blackburn_30361032,1067.783325,1079.664490,1027.531982,1036.002106,Small Car,100,Petrol cars,651.68,639.8,542.97,551.39,543.34
1,blackburn_30361032,1975.399071,1986.786041,528.676117,538.260757,Small Car,109,Petrol cars,651.68,639.8,542.97,551.39,543.34
2,blackburn_30361032,1979.620247,1991.101990,514.587952,524.379181,Small Car,112,Petrol cars,651.68,639.8,542.97,551.39,543.34
3,blackburn_30361032,1824.158295,1833.719086,641.948860,650.265575,Small Car,79,Petrol cars,651.68,639.8,542.97,551.39,543.34
4,blackburn_30361032,2237.735962,2248.019989,209.058350,219.792877,Small Car,110,Petrol cars,651.68,639.8,542.97,551.39,543.34


## Ratio of AADT to each Vehicle Category

In [16]:
for df in dfs:

  reciprocal_vehicle_count = 1 / len(df)

  if 'aadt' in df:
    aadt = df.iloc[0]['aadt']

    print("image_id: {}, aadt: {}".format(df.iloc[0]['image_id'], aadt))
    df['aadt_vehicles'] = aadt * reciprocal_vehicle_count

dfs[0].head()

image_id: blackburn_30361032, aadt: 651.68
image_id: blackburn_30361033, aadt: 663.92
image_id: havering_m25_5790a, aadt: 1065.38
image_id: havering_m25_5790b, aadt: 1071.64
image_id: hounslow_m4_2188a, aadt: 1198.62
image_id: hounslow_m4_2188b, aadt: 1375.54
image_id: trafford_m60_9083a, aadt: 973.21
image_id: trafford_m60_9086b, aadt: 1005.7
image_id: luton_m1_2557a, aadt: 588.1
image_id: luton_m1_2557b, aadt: 586.98


,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,aadt_vehicles
0,blackburn_30361032,1067.783325,1079.664490,1027.531982,1036.002106,Small Car,100,Petrol cars,651.68,639.8,542.97,551.39,543.34,50.129231
1,blackburn_30361032,1975.399071,1986.786041,528.676117,538.260757,Small Car,109,Petrol cars,651.68,639.8,542.97,551.39,543.34,50.129231
2,blackburn_30361032,1979.620247,1991.101990,514.587952,524.379181,Small Car,112,Petrol cars,651.68,639.8,542.97,551.39,543.34,50.129231
3,blackburn_30361032,1824.158295,1833.719086,641.948860,650.265575,Small Car,79,Petrol cars,651.68,639.8,542.97,551.39,543.34,50.129231
4,blackburn_30361032,2237.735962,2248.019989,209.058350,219.792877,Small Car,110,Petrol cars,651.68,639.8,542.97,551.39,543.34,50.129231


In [17]:
dfs[4]

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,aadt_vehicles
0,hounslow_m4_2188a,0,10,0,10,Small Car,100,Petrol cars,1198.62,1066.96,626.38,681.01,628.59,1198.62


## Calculate GHG Emissions 


In [18]:
total_emissions = []

for df in dfs:
  la_name_id = df.iloc[0]['image_id']
  ghg_emissions = 0
  LENGTH = 0

  print(la_name_id)

  if 'aadt' in df:

    aadt = df.iloc[0]['aadt']
    cars_and_taxis = df.iloc[0]['cars_and_taxis']
    buses_and_coaches = df.iloc[0]['buses_and_coaches']
    lgvs = df.iloc[0]['lgvs']
    all_hgvs = df.iloc[0]['all_hgvs']

    if la_name_id.find('blackburn') != -1:
      LENGTH = BLACKBURN_ROAD_LENGTH

    elif la_name_id.find('luton') != -1:
      LENGTH = LUTON_ROAD_LENGTH

    elif la_name_id.find('hounslow') != -1:
      LENGTH = HOUNSLOW_ROAD_LENGTH

    elif la_name_id.find('havering') != -1:
      LENGTH = HAVERING_ROAD_LENGTH

    elif la_name_id.find('trafford') != -1:
      LENGTH = TRAFFORD_ROAD_LENGTH


    # VEHICLE KM TRAVELLED (km)
    aadt_vehicle_km_travel = LENGTH * df.iloc[0]['aadt'] * 365
    cars_and_taxis_vehicle_km_travel = LENGTH * df.iloc[0]['cars_and_taxis'] * 365
    buses_and_coaches_vehicle_km_travel = LENGTH * df.iloc[0]['buses_and_coaches'] * 365
    lgvs_vehicle_km_travel = LENGTH * df.iloc[0]['lgvs'] * 365
    all_hgvs_vehicle_km_travel = LENGTH * df.iloc[0]['all_hgvs'] * 365



    # SPECIFIC FUEL CONSUMPTION (km/litre)
    aadt_vehicle_km_litre = VEHICLE_KM_PER_LITRE_MAPPING['aadt']
    cars_and_taxis_vehicle_km_litre = VEHICLE_KM_PER_LITRE_MAPPING['cars_and_taxis']
    buses_and_coaches_vehicle_km_litre = VEHICLE_KM_PER_LITRE_MAPPING['buses_and_coaches']
    lgvs_vehicle_km_litre = VEHICLE_KM_PER_LITRE_MAPPING['lgvs']
    all_hgvs_vehicle_km_litre = VEHICLE_KM_PER_LITRE_MAPPING['all_hgvs']


    # LITRES USED (litres)
    aadt_litres = aadt_vehicle_km_travel / aadt_vehicle_km_litre # litres
    cars_and_taxis_litres = aadt_vehicle_km_travel / cars_and_taxis_vehicle_km_litre # litres
    buses_and_coaches_litres = buses_and_coaches_vehicle_km_travel / buses_and_coaches_vehicle_km_litre # litres
    lgvs_litres = lgvs_vehicle_km_travel / lgvs_vehicle_km_litre # litres
    all_hgvs_litres = all_hgvs_vehicle_km_travel / all_hgvs_vehicle_km_litre # litres


    # EMISSIONS FACTORS (kg CO2)
    aadt_emissions_factor = df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == VEHICLE_EMISSIONS_FACTORS_MAPPING['aadt'], 'Total'].values[0] # kg co2
    cars_and_taxis_emissions_factor = df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == VEHICLE_EMISSIONS_FACTORS_MAPPING['cars_and_taxis'], 'Total'].values[0] # kg co2
    buses_and_coaches_emissions_factor = df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == VEHICLE_EMISSIONS_FACTORS_MAPPING['buses_and_coaches'], 'Total'].values[0] # kg co2
    lgvs_emissions_factor = df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == VEHICLE_EMISSIONS_FACTORS_MAPPING['lgvs'], 'Total'].values[0] # kg co2
    all_hgvs_emissions_factor = df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == VEHICLE_EMISSIONS_FACTORS_MAPPING['all_hgvs'], 'Total'].values[0] # kg co2


    # GHG EMISSIONS (kg CO2)
    aadt_emissions = aadt_emissions_factor * aadt_litres
    cars_and_taxis_emissions = cars_and_taxis_emissions_factor * cars_and_taxis_litres
    buses_and_coaches_emissions = buses_and_coaches_emissions_factor * buses_and_coaches_litres
    lgvs_emissions = lgvs_emissions_factor * lgvs_litres
    all_hgvs_emissions = all_hgvs_emissions_factor * all_hgvs_litres

    # TOTAL EMISSIONS (kg CO2)
    ghg_emissions = np.round(cars_and_taxis_emissions + buses_and_coaches_emissions + lgvs_emissions + all_hgvs_emissions, 1)

    ghg_emissions = ghg_emissions * KG_TO_KT

    print("LA Count Site: {}, AADT Prediction: {}, GHG Emissions Prediction: {}".format(la_name_id, aadt, ghg_emissions))

    save_float_to_csv(ghg_emissions, 'ghg_emissions', image_id=la_name_id, file_name=GHG_EMISSIONS_ROOT_PATH+'ghg_emissions_'+la_name_id+'.csv')
    total_emissions.append((la_name_id, ghg_emissions))

blackburn_30361032
LA Count Site: blackburn_30361032, AADT Prediction: 651.68, GHG Emissions Prediction: 186.8520582
blackburn_30361033
LA Count Site: blackburn_30361033, AADT Prediction: 663.92, GHG Emissions Prediction: 189.34023309999998
havering_m25_5790a
LA Count Site: havering_m25_5790a, AADT Prediction: 1065.38, GHG Emissions Prediction: 234.224956
havering_m25_5790b
LA Count Site: havering_m25_5790b, AADT Prediction: 1071.64, GHG Emissions Prediction: 233.6200663
hounslow_m4_2188a
LA Count Site: hounslow_m4_2188a, AADT Prediction: 1198.62, GHG Emissions Prediction: 187.4972095
hounslow_m4_2188b
LA Count Site: hounslow_m4_2188b, AADT Prediction: 1375.54, GHG Emissions Prediction: 194.4803105
trafford_m60_9083a
LA Count Site: trafford_m60_9083a, AADT Prediction: 973.21, GHG Emissions Prediction: 307.124303
trafford_m60_9086b
LA Count Site: trafford_m60_9086b, AADT Prediction: 1005.7, GHG Emissions Prediction: 313.9280958
luton_m1_2557a
LA Count Site: luton_m1_2557a, AADT Predicti